# **ETL for Credit Card Churn Analysis**

## Objectives

* This notebook loads the dataset required for DataVisualisation.ipynb. It includes an initial data inspection and a data quality summary. Any identified data issues will be addressed, followed by column encoding and feature engineering to support the visualisation and analysis stage.

## Inputs

* kaggle based credit-card-customers dataset (sakshigoyal7/credit-card-customers) [URL link ](https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers)

## Outputs

* Cleansed file of current & churned credit card customers & there attributes. Located in 'dataFiles/cleanData' sub directory. 

## Additional Comments

* This will only contain the ETL elements of this process. Please see notebook DataVisualisation.ipynb for visualisation and hypothesis analysis step.  



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

print("Current Working Directory:", current_dir)   
  


Current Working Directory: c:\Users\ryan_\VS-code-projects\CreditCardChurn\jupyter_notebooks


We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))

print("You set a new current directory")


You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\ryan_\\VS-code-projects\\CreditCardChurn'

# Section 1 - Pull the dataset to local system

In [4]:
dwpath = os.path.join(current_dir, 'dataFiles', 'rawData')
outpath = os.path.join(current_dir,'dataFiles', 'cleanData')
print("Data Download Path:", dwpath)    
print("Data Cleansed Path:", outpath)

Data Download Path: c:\Users\ryan_\VS-code-projects\CreditCardChurn\dataFiles\rawData
Data Cleansed Path: c:\Users\ryan_\VS-code-projects\CreditCardChurn\dataFiles\cleanData


This section aims to pull the dataset from Kaggle via their API. 
(an account was previously created on website and api json file was generated and saved on my system)

In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

api.dataset_download_files('sakshigoyal7/credit-card-customers', path = dwpath, unzip= True)



Dataset URL: https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers


> **Note:**  
> This step uses the Kaggle API and requires a valid `kaggle.json` credentials file to be configured on the local machine.  
> Without this file, the download step will fail on a new or clean environment.


In [6]:
#checking the file downnloaded is accessible and has records. 
import pandas as pd
import numpy as np
df_raw = pd.read_csv(os.path.join(dwpath,'BankChurners.csv'))

df_raw.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


---

# Section 2 - Initial Data Inspection 


This section evaluates the dataset structure and quality:
- shape and column names
- data types
- missing values
- duplicate rows
- target distribution (churn vs non-churn) - a key measure that is of focus in the business requirements

Based of these results, it will lead the level of data cleaning required in later steps. 

In [7]:
# Get a view of basic shape and structure of the data

print("Shape:", df_raw.shape)
display(df_raw.head(3))
display(df_raw.tail(3))


Shape: (10127, 23)


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998


,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.99788,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.99671,0.003294
10126,714337233,Attrited Customer,43,F,2,Graduate,Married,Less than $40K,Silver,25,...,10388.0,1961,8427.0,0.703,10294,61,0.649,0.189,0.99662,0.003377


From the head() & tail() samples, it's clear that records are populated at the start and end of the file.

In [8]:
#Showing columns names found in raw dataset 
df_raw.columns 


Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
      dtype='object')

Looking at the column names, values, and types, I decided to remove the following columns because they’re not business-facing fields. They appear to be artifacts from a Naive Bayes classifier output and could confuse downstream EDA:

- Naive_Bayes_Classifier_..._1

- Naive_Bayes_Classifier_..._2 

In [9]:
# Dropping the two odd columns 
ODD_COLS = [
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",
]

# Dropping only if present
df_raw = df_raw.drop(columns=[c for c in ODD_COLS if c in df_raw.columns])

#checking columns after column drop
df_raw.columns 

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')

Above confirms the columns have been dropped. 

In [10]:
# Missing values check
missing = df_raw.isna().sum().sort_values(ascending=False)
missing = missing[missing > 0]
missing


Series([], dtype: int64)

Great news no obvious missing values.

In [11]:
# Duplicate rows check
dup_count = df_raw.duplicated().sum()
print("Duplicate rows:", dup_count)


Duplicate rows: 0


In [12]:
# Target distribution check
TARGET_COL = "Attrition_Flag"
target_counts = df_raw[TARGET_COL].value_counts(dropna=False)
target_pct = df_raw[TARGET_COL].value_counts(normalize=True, dropna=False).round(4) * 100  #translate into precentages 

display(pd.DataFrame({"count": target_counts, "pct": target_pct}))


,count,pct
Attrition_Flag,,
Existing Customer,8500,83.93
Attrited Customer,1627,16.07


In [13]:
#using descibe() to check numerical and categorical columns (aka sanity check)
display(df_raw.describe(include="number").T)
display(df_raw.describe(include="object").T)

,count,mean,std,min,25%,50%,75%,max
CLIENTNUM,10127.0,7.391776e+08,3.690378e+07,708082083.0,7.130368e+08,7.179264e+08,7.731435e+08,8.283431e+08
Customer_Age,10127.0,4.632596e+01,8.016814e+00,26.0,4.100000e+01,4.600000e+01,5.200000e+01,7.300000e+01
Dependent_count,10127.0,2.346203e+00,1.298908e+00,0.0,1.000000e+00,2.000000e+00,3.000000e+00,5.000000e+00
Months_on_book,10127.0,3.592841e+01,7.986416e+00,13.0,3.100000e+01,3.600000e+01,4.000000e+01,5.600000e+01
Total_Relationship_Count,10127.0,3.812580e+00,1.554408e+00,1.0,3.000000e+00,4.000000e+00,5.000000e+00,6.000000e+00
Months_Inactive_12_mon,10127.0,2.341167e+00,1.010622e+00,0.0,2.000000e+00,2.000000e+00,3.000000e+00,6.000000e+00
Contacts_Count_12_mon,10127.0,2.455317e+00,1.106225e+00,0.0,2.000000e+00,2.000000e+00,3.000000e+00,6.000000e+00
Credit_Limit,10127.0,8.631954e+03,9.088777e+03,1438.3,2.555000e+03,4.549000e+03,1.106750e+04,3.451600e+04
Total_Revolving_Bal,10127.0,1.162814e+03,8.149873e+02,0.0,3.590000e+02,1.276000e+03,1.784000e+03,2.517000e+03
Avg_Open_To_Buy,10127.0,7.469140e+03,9.090685e+03,3.0,1.324500e+03,3.474000e+03,9.859000e+03,3.451600e+04


,count,unique,top,freq
Attrition_Flag,10127,2,Existing Customer,8500
Gender,10127,2,F,5358
Education_Level,10127,7,Graduate,3128
Marital_Status,10127,4,Married,4687
Income_Category,10127,6,Less than $40K,3561
Card_Category,10127,4,Blue,9436


### Data Quality Summary (as of 21/01/2026)
Steps:
- Missing values: No reported missing values 
- Duplicates:  No reported duplicates 
- Target balance: Attrited Customers represent 16.07% of the dataset (1,627 out of 8,500).
- Sanity check on the data using 'describe()' to see ranges on columns values are credible (e.g. Customer_Age min_value= 26 is crediable in this situation ) 




---

Decision: As it stands the data looks ok, so right now there is no need for me to perform any lengthy data cleaning. 

# Section 3 -Transform - Cleaning data 

The below section will go over the required data cleansing steps highlighted in the previous sections or during exploration during DataVisualisation's notebook. 
Before starting these are the typical high level stages I would perform. 
- Removing duplicates (if any)
- Handling missing values (if any)
- Standardising column names (per reqs or personal preference)

In [14]:
#move away from DF_raw 
#start with renaming columns based on PascalCase standard  to improve readability and consistency going forward.”

df = df_raw.copy()

#renaming columns to PascalCase via for loop
df.columns = [
    "".join(word.capitalize() for word in c.strip().split("_"))
    for c in df.columns
]
#checking renaming of columns 
print("Original Columns : ")
print(df_raw.columns)

print("Renamed Columns : ")
df.columns


Original Columns : 
Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')
Renamed Columns : 


Index(['Clientnum', 'AttritionFlag', 'CustomerAge', 'Gender', 'DependentCount',
       'EducationLevel', 'MaritalStatus', 'IncomeCategory', 'CardCategory',
       'MonthsOnBook', 'TotalRelationshipCount', 'MonthsInactive12Mon',
       'ContactsCount12Mon', 'CreditLimit', 'TotalRevolvingBal',
       'AvgOpenToBuy', 'TotalAmtChngQ4Q1', 'TotalTransAmt', 'TotalTransCt',
       'TotalCtChngQ4Q1', 'AvgUtilizationRatio'],
      dtype='object')

Above updates the column names to the format of PascalCase. This imporves readability and ensures consistency going forward.  
As there wasn't any identified missing values from our previous checks, I'll move on to column encoding. 

# Section 4 - Transform - Encoding of Columns 

Categorical variables were encoded to support downstream analysis and plotting:
- Ordinal categories were mapped manually where a logical hierarchy exists.
- Nominal categories were encoded using categorical codes, with the understanding that these codes represent labels rather than ordered values.

Special handling was applied to `IncomeCategory`, where “Unknown” values were converted to nulls to prevent them from being misinterpreted as meaningful income levels.


Below converts `AttritionFlag` into a numeric churn indicator for analysis:
- Existing Customer → 0
- Attrited Customer → 1

The new column will be called `ChurnFlag`
This has been done as it will support easier business understanding of analysis performed in the business/peer notebook. 


In [15]:

TGT_COL = "AttritionFlag"
CHURN_COL = "ChurnFlag"

target_map = {
    "Existing Customer": 0,
    "Attrited Customer": 1
}

df[CHURN_COL] = df[TGT_COL].map(target_map)

# Validation of new churn flag 
display(df[[TGT_COL, CHURN_COL]].head(22))
#checking nulls in churn flag column
print("Nulls in churn flag:", df[CHURN_COL].isna().sum())
df[CHURN_COL].value_counts(dropna=False)

,AttritionFlag,ChurnFlag
0,Existing Customer,0
1,Existing Customer,0
2,Existing Customer,0
3,Existing Customer,0
4,Existing Customer,0
5,Existing Customer,0
6,Existing Customer,0
7,Existing Customer,0
8,Existing Customer,0
9,Existing Customer,0


Nulls in churn flag: 0


ChurnFlag
0    8500
1    1627
Name: count, dtype: int64

Next the column `Gender` will be encoded `GenderEnc`. 

In [16]:
if "Gender" in df.columns:
    df["GenderEnc"] = df["Gender"].map({"M": 1, "F": 0})
    print("Encoded Gender -> GenderEnc")
df["GenderEnc"].value_counts(dropna=False)

Encoded Gender -> GenderEnc


GenderEnc
0    5358
1    4769
Name: count, dtype: int64

Next column to be encoded is 'IncomeCategory'. 

In [17]:
df['IncomeCategory'].unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +', 'Unknown'], dtype=object)

From the possible range of values, we can see we have a Unknown entry. 
Right I’d assume that Unknown could mean: the customer didnt provide an answer, data was not capatured, specific information was lost. 

Because the rest of the categorys have an implied order `60K-80k` is smaller than `80k-120k` I want to ensure that the encode column doesnt mislead in any downstream reports or vis's. As its likely someone could infer that from the column and muddy actions like means,medians and trending. 


In [18]:
income_order = {
    "Less than $40K": 0,
    "$40K - $60K": 1,
    "$60K - $80K": 2,
    "$80K - $120K": 3,
    "$120K +": 4,
    "Unknown": np.nan
}
df["IncomeCatEnc"] = df["IncomeCategory"].map(income_order)

df["IncomeCatEnc"].value_counts(dropna=False)

IncomeCatEnc
0.0    3561
1.0    1790
3.0    1535
2.0    1402
NaN    1112
4.0     727
Name: count, dtype: int64

Next will check the possible values in the other columns i'd like to encode. 

In [19]:

for col in ["EducationLevel", "MaritalStatus", "CardCategory"]:
    print(df[col].value_counts(dropna=False))


EducationLevel
Graduate         3128
High School      2013
Unknown          1519
Uneducated       1487
College          1013
Post-Graduate     516
Doctorate         451
Name: count, dtype: int64
MaritalStatus
Married     4687
Single      3943
Unknown      749
Divorced     748
Name: count, dtype: int64
CardCategory
Blue        9436
Silver       555
Gold         116
Platinum      20
Name: count, dtype: int64


For these columns I am happy to make use encoding where there is no assumption to number hierarchy. So Pandas will assign a value to each encoding case. 

In [20]:
for col in ["EducationLevel", "MaritalStatus", "CardCategory"]:
    df[col + "Enc"] = df[col].astype("category").cat.codes

In [21]:
#checking encoded columns value counts
for col in ["EducationLevelEnc", "MaritalStatusEnc", "CardCategoryEnc"]:
    print(df[col].value_counts(dropna=False))

EducationLevelEnc
2    3128
3    2013
6    1519
5    1487
0    1013
4     516
1     451
Name: count, dtype: int64
MaritalStatusEnc
1    4687
2    3943
3     749
0     748
Name: count, dtype: int64
CardCategoryEnc
0    9436
3     555
1     116
2      20
Name: count, dtype: int64


The counts match the source column counts from previous step. 

---


# Section 5 - Feature engineering

### Feature Engineering

Additional features were created to better capture customer behaviour and improve analytical insight:

- **Average Transaction Value** was derived to normalise transaction behaviour across customers.
- **Inactivity Flags** were introduced to identify customers with prolonged periods of inactivity.
- **Utilisation Bands** were created by binning utilisation ratios into interpretable ranges, improving readability in visualisations and summary analysis.

These engineered features are designed to support the exploratory and visual analysis stage of the project, particularly in understanding churn behaviour and customer engagement patterns.


In [22]:
df.describe()

,Clientnum,CustomerAge,DependentCount,MonthsOnBook,TotalRelationshipCount,MonthsInactive12Mon,ContactsCount12Mon,CreditLimit,TotalRevolvingBal,AvgOpenToBuy,...,TotalTransAmt,TotalTransCt,TotalCtChngQ4Q1,AvgUtilizationRatio,ChurnFlag,GenderEnc,IncomeCatEnc,EducationLevelEnc,MaritalStatusEnc,CardCategoryEnc
count,1.012700e+04,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,...,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,9015.000000,10127.000000,10127.000000,10127.000000
mean,7.391776e+08,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,...,4404.086304,64.858695,0.712222,0.274894,0.160660,0.470919,1.342984,3.096574,1.463415,0.179816
std,3.690378e+07,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,...,3397.129254,23.472570,0.238086,0.275691,0.367235,0.499178,1.356455,1.834812,0.737808,0.693039
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,...,510.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.130368e+08,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,...,2155.500000,45.000000,0.582000,0.023000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000
50%,7.179264e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,...,3899.000000,67.000000,0.702000,0.176000,0.000000,0.000000,1.000000,3.000000,1.000000,0.000000
75%,7.731435e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,...,4741.000000,81.000000,0.818000,0.503000,0.000000,1.000000,3.000000,5.000000,2.000000,0.000000
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,...,18484.000000,139.000000,3.714000,0.999000,1.000000,1.000000,4.000000,6.000000,3.000000,3.000000


In [23]:

# Avg transaction value
df["AvgTransactionValue"] = df["TotalTransAmt"] / df["TotalTransCt"].replace(0, np.nan)

# Inactivity flag
df["Inactive3plusFlag"] = (df["MonthsInactive12Mon"] >= 3).astype(int)


df["UtilizationBand"] = pd.cut(
    df["AvgUtilizationRatio"],
    bins=[-0.001, 0.0, 0.25, 0.5, 0.75, 1.0],
    labels=["0", "0-0.25", "0.25-0.5", "0.5-0.75", "0.75-1.0"]
)



---

# Section 6 - Saving clean data to file

In [24]:

print("Target file path: ", outpath) 
targetFile = os.path.join(outpath, 'cleaned_credit_card_churn_data.csv')
print("Target file path and name:", targetFile)
#copy etl finished version to new object
df_clean = df.copy()
#Index is excluded because it is not a meaningful business field and would add noise to downstream analysis.
df_clean.to_csv(targetFile, index=False)
print("Cleansed file created")



Target file path:  c:\Users\ryan_\VS-code-projects\CreditCardChurn\dataFiles\cleanData
Target file path and name: c:\Users\ryan_\VS-code-projects\CreditCardChurn\dataFiles\cleanData\cleaned_credit_card_churn_data.csv
Cleansed file created
